In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rp

In [3]:
base = "http://ufo-api.herokuapp.com/api/sightings/location/near?"
Ontario_lat_max = 56.800878
Ontario_lat_min = 41.771312
Ontario_lng_max = -75.251467
Ontario_lng_min = -95.281572

lats = np.random.uniform(low=Ontario_lat_min, high=Ontario_lat_max, size=500)
lngs = np.random.uniform(low=Ontario_lng_min, high=Ontario_lng_max, size=500)


In [4]:
len(lats), len(lngs)

(500, 500)

In [5]:
city = []
shape = []
state = []
date = []
duration = []
lats_ON = []
lngs_ON = []
for i in np.arange(0,len(lats)):
    try:
        responce = rp.get(f"{base}lat={lats[i]}&lon={lngs[i]}&limit=1").json()
        if responce["sightings"][0]["obj"]["state"] == "ON":
            city.append(responce["sightings"][0]["obj"]["city"])
            shape.append(responce["sightings"][0]["obj"]["shape"])
            state.append(responce["sightings"][0]["obj"]["state"])
            date.append(responce["sightings"][0]["obj"]["date"])
            duration.append(responce["sightings"][0]["obj"]["duration"])
            lats_ON.append(responce["sightings"][0]["obj"]["loc"][0])
            lngs_ON.append(responce["sightings"][0]["obj"]["loc"][1])
    except Exception as e:
        print("could not find")
print("finished")

finished


In [47]:
Combined = pd.DataFrame({"lat":lats_ON,"lng":lngs_ON,"city":city,"shape":shape,"state":state,"date":date,"duration":duration})
Combined.head()

,lat,lng,city,shape,state,date,duration
0,-82.428349,52.923989,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes
1,-81.146137,44.144277,Walkerton (Canada),Rectangle,ON,2010-01-30T18:45:00.000Z,1 min
2,-82.214605,45.756312,Manitoulin Island (Canada),Fireball,ON,2011-10-22T03:30:00.000Z,5 sec
3,-82.428349,52.923989,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes
4,-82.428349,52.923989,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes


In [21]:
x = rp.get("http://ontariobeerapi.ca/beers/").json()


In [32]:
name = []
price = []

for i in np.arange(0,len(x)):
    name.append(x[i]["name"])
    price.append(x[i]["price"])

In [45]:
y = pd.DataFrame({"Name":name, "Price":price})

y.sort_values("Price",a)

,Name,Price
635,Millennium Buzz Hemp Beer,1.85
283,Mojo Shot Melon Ball,10.00
381,Beer League Lager,10.00
435,Welly Re-booted Vol 4,10.45
439,Revenge Ginger Red IPA,10.50
412,Laker Light,10.50
411,Laker Lager,10.50
402,PC Down Under Amber,10.75
39,Blue Light,10.95
116,Flying Monkeys Amber Ale,10.95
